In [9]:
import os
import pandas as pd
data_dir =  'data/cropped/'
data_per = os.listdir(data_dir)

In [10]:
def max_frame(data_per):
    max_frames = 0
    for person in data_per:
        words_dir = f'{data_dir}/{person}'
        Id = os.listdir(words_dir)
        for id in Id :
            id_dir = f'{data_dir}/{person}/{id}'
            num = os.listdir(id_dir)
            for n  in num:
                #print("n is", n)
                words_dir_2 = f'{data_dir}/{person}/{id}/{n}/crop_face'
                frames = os.listdir(words_dir_2)
                #print(len(frames))
                max_frames = max(len(frames), max_frames)
    return max_frames

In [11]:
max_frame(data_per)

11

In [13]:
from keras.preprocessing import image
import numpy as np

for person in data_per:
    #phrases_dir = f'{data_dir}/{person}/phrases'
    words_dir = f'{data_dir}/{person}'
    Id = os.listdir(words_dir)
    for sentence in Id :
        id_dir = f'{data_dir}/{person}/{sentence}'
        num = os.listdir(id_dir)
        for n  in num:
            print("person: {}, sentence: {}, n: {} ".format(person, sentence, n))
            words_dir_2 = f'{data_dir}/{person}/{sentence}/{n}/crop_face'
            frames = os.listdir(words_dir_2)
            x_data_ = np.empty(3)
            for frame in frames:
                img_path = f'{words_dir_2}/{frame}'
                
                # 이미지 하나 당 flatten 해주어서 12288 차원으로 만들어 줌
                img = image.load_img(img_path, target_size=(64, 64))
                img_tensor = image.img_to_array(img) #tensor로 type 변경
                img_tensor = img_tensor.flatten() # flatten: 12288 차원
                img_tensor /= 255. # scaling 
                img_tensor = np.expand_dims(img_tensor, axis = 0) # (1, 12288) 차원
                
                # frame 별 묶기
                if frame == 'face_001.jpg':
                    x_data_ = img_tensor
                else:
                    x_data_ = np.concatenate([x_data_, img_tensor], axis = 0)
                
                print(x_data_.shape)
            '''
            # 이미지 전체 통합시키기
            x_data_ = np.expand_dims(x_data_, axis = 0)
            if n == '01':
                x_data = x_data_
            else:
                x_data = np.concatenate([x_data, x_data_], axis = 0)
            '''
            # 이미지 전체 통합시키기
            x_data_ = np.expand_dims(x_data_, axis = 0)  #[1, frame, image_size]
            if (n == '01' and sentence == '01' and person == 'F01'):
                if x_data_.shape[0] == max_frame(data_per):
                    x_data = np.concatenate([x_data, x_data_], axis = 1) 
                else: 
                    pad = max_frame(data_per) - x_data_.shape[1]
                    print("pad for ",pad)
                    zero_pad = np.zeros((1, pad, 12288))
                    print(zero_pad.shape)
                    print(x_data_.shape)
                    data = np.concatenate([x_data_, zero_pad], axis = 1) #padding
                    x_data = data
            else:
                if x_data_.shape[0] == max_frame(data_per):
                    x_data = np.concatenate([x_data, x_data_], axis = 1) 
                else: 
                    pad = max_frame(data_per) - x_data_.shape[1]
                    print("pad for ",pad)
                    zero_pad = np.zeros((1, pad, 12288))
                    #print(zero_pad.shape)
                    #print(x_data_.shape)
                    data = np.concatenate([x_data_, zero_pad], axis = 1) #padding
                    #print(data.shape)
                    x_data = np.concatenate([x_data, data], axis = 0)
            print(x_data.shape)

person: F01, sentence: 01, n: 01 
(1, 12288)
(2, 12288)
(3, 12288)
(4, 12288)
(5, 12288)
(6, 12288)
(7, 12288)
(8, 12288)
(9, 12288)
(10, 12288)
pad for  2
(1, 2, 12288)
(1, 10, 12288)
(1, 12, 12288)
person: F01, sentence: 01, n: 02 
(1, 12288)
(2, 12288)
(3, 12288)
(4, 12288)
(5, 12288)
(6, 12288)
(7, 12288)
pad for  5
(2, 12, 12288)
person: F01, sentence: 01, n: 03 
(1, 12288)
(2, 12288)
(3, 12288)
(4, 12288)
(5, 12288)
(6, 12288)
(7, 12288)
(8, 12288)
(9, 12288)
(10, 12288)
pad for  2
(3, 12, 12288)
person: F01, sentence: 01, n: 04 
(1, 12288)
(2, 12288)
(3, 12288)
(4, 12288)
(5, 12288)
(6, 12288)
(7, 12288)
(8, 12288)
(9, 12288)
pad for  3
(4, 12, 12288)
person: F01, sentence: 01, n: 05 
(1, 12288)
(2, 12288)
(3, 12288)
(4, 12288)
(5, 12288)
(6, 12288)
(7, 12288)
(8, 12288)
(9, 12288)
(10, 12288)
(11, 12288)
pad for  1
(5, 12, 12288)
person: F01, sentence: 01, n: 06 
(1, 12288)
(2, 12288)
(3, 12288)
(4, 12288)
(5, 12288)
(6, 12288)
(7, 12288)
(8, 12288)
(9, 12288)
pad for  3
(6, 12

In [ ]:
class DataLoader(Sequence):
    def __init__(self, data_list, batch_size):
        super(DataLoader, self). __init__()
        # 데이터셋의 전처리를 해주는 부분
    def __getitem__(self, index): 
        # 데이터셋에서 특정 1개의 샘플을 가져오는 함수
        # index 값에 따라 데이터를 반환하는 함수
        return self.data_list[index]
    def __len__(self):
        #  데이터셋의 길이 함수 (총 샘플의 수를 적어주는 부분)
        return len(self.data_list)

In [ ]:
if __name__ = "__main__":
    model = Model()
    data_list = getDataList()
    batch_size = 16
    
    data_loader = DataLoader(data_list, batch_size)
    model.fit(data_loader)